In [1]:
#current_xml = "read_book.bpmn"
#current_xml = "read_book_vertauscht.bpmn"
#current_xml = "read_book_xor.bpmn"
current_xml = "read_book_and.bpmn"
#current_xml = "assignment1.bpmn"

In [ ]:
import xml.etree.ElementTree as ET

def get_default_namespace(line):
    line_list = line.split()
    for y in line_list:
        if y.find("xmlns=")> -1:
            pos = (y.find('"')) + 1
            namespace = (y[pos:-1])
            return namespace

        
def get_vorg_id_of_task(task, process):
    incoming_flow_id = task.find(ns+"incoming")
    incoming_flow_id = incoming_flow_id.text
    source_ref = ""
    for child in process:
        tagname = (child.tag.split('}')[1])
        if tagname == "sequenceFlow":
            child_id = child.attrib["id"]
            if child_id == incoming_flow_id:
                source_ref = child.attrib["sourceRef"]
    return source_ref

In [7]:
#Annahmen:
# Es gibt genau 1 Startereignnis
# Es gibt genau 1 Endereignis
# Alle BPMN Elemente befinden sich auf der direkten Unterebene von <process>

t_counter = 0
p_counter = 0
net = [] # [{transnum, transname, in, out}, ...]

with open(current_xml, "r") as f:
    ns = get_default_namespace(f.readline())
    ns = "{" + ns + "}"
    tree = ET.parse(current_xml)
    root = tree.getroot()
    process = root[0]
    
    for child in process:
        tagname = (child.tag.split('}')[1])
        
        if tagname == "startEvent": # place, trans und place erstellen
            net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"start", "in":{p_counter}, "out":{p_counter+1}}
            net.append(net_elem)
            p_counter += 1
            t_counter += 1
        
        elif "task" in tagname or "Task" in tagname: # trans und place erstellen
            net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":child.attrib["name"], "in":{"dummy"}, "out":{p_counter+1}} 
            p_counter += 1
            t_counter += 1
            vorg_id = get_vorg_id_of_task(child, process)
            vorg_place = set()
            for trans in net:
                if trans["id"] == vorg_id:
                    vorg_place = trans["out"]
                    net_elem["in"] = vorg_place
            net.append(net_elem)
            
        elif tagname == "endEvent": #trans und place erstellen
            net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"end", "in":{"dummy2"}, "out":{p_counter+1}}
            net.append(net_elem)
            p_counter += 1
            t_counter += 1
            
        elif tagname == "parallelGateway":
            if child.attrib["gatewayDirection"] == "Diverging":# trans und 2 out places machen
                net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"silent_and_div", "in":{"dummy"}, "out":{p_counter+1, p_counter+2}}
                net.append(net_elem)
                p_counter += 2
                t_counter += 1
            
            if child.attrib["gatewayDirection"] == "Converging": #trans und place machen
                net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"silent_and_con", "in":{"dummy"}, "out":{p_counter+1}} 
                net.append(net_elem)
                p_counter += 1
                t_counter += 1
        
        elif tagname == "exclusiveGateway":
            if child.attrib["gatewayDirection"] == "Diverging": # 2 trans mit je 1 place machen
                net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"silent_xor_div_1", "in":{"dummy"}, "out":{p_counter+1}}
                net.append(net_elem)
                p_counter += 1
                t_counter += 1
                net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"silent_xor_div_2", "in":{"dummy"}, "out":{p_counter+1}}
                net.append(net_elem)
                p_counter += 1
                t_counter += 1
            
            if child.attrib["gatewayDirection"] == "Diverging": #2 trans mit dem gleiche out place machen
                net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"silent_xor_con_1", "in":{"dummy"}, "out":{p_counter+1}}
                net.append(net_elem)
                t_counter += 1
                #p_counter darf man hier nicht erhöhen
                net_elem = {"id": child.attrib["id"], "transnum":t_counter, "transname":"silent_xor_con_2", "in":{"dummy"}, "out":{p_counter+1}}
                net.append(net_elem)
                p_counter += 1
                t_counter += 1
            
                
                
                
for net_elem in net:
    print(net_elem)
    
    

    






{'id': 'sid-06AE33F6-621B-46B6-9F10-8C532D527FA0', 'transnum': 0, 'transname': 'open Book', 'in': {'dummy'}, 'out': {1}}
{'id': 'sid-09A3F697-4661-43B9-B341-4227406D812E', 'transnum': 1, 'transname': 'read Book', 'in': {'dummy'}, 'out': {2}}
{'id': 'sid-4B5F6797-5F42-417B-AA5C-D2D4477005C8', 'transnum': 2, 'transname': 'play with iPhone', 'in': {'dummy'}, 'out': {3}}
{'id': 'sid-F0C9382C-94E0-4DE4-B4C4-405CF5EDB6A0', 'transnum': 3, 'transname': 'close Book', 'in': {'dummy'}, 'out': {4}}
{'id': 'sid-BF7A296E-5C1A-4C1B-82EF-E5E3C1E5EDBC', 'transnum': 4, 'transname': 'end', 'in': {'dummy2'}, 'out': {5}}
{'id': 'sid-0607E960-A735-47DC-970F-0FDA3E9CD192', 'transnum': 5, 'transname': 'start', 'in': {5}, 'out': {6}}
{'id': 'sid-3A5DEFCB-DC08-4562-92A6-AC3A546B798D', 'transnum': 6, 'transname': 'silent_and_div', 'in': {'dummy'}, 'out': {8, 7}}
{'id': 'sid-AF74B9C7-A0AF-4247-96B4-3B750B7060C7', 'transnum': 7, 'transname': 'silent_and_con', 'in': {'dummy'}, 'out': {9}}
